In [ ]:
# path to our tables in the silver layer
silver_tables_path = "abfss://3201caea-2820-458c-8915-9fe2f28ba2a7@onelake.dfs.fabric.microsoft.com/4e18f678-60fe-4988-82c1-2d51453db35e/Tables"

StatementMeta(, , , Waiting, )

In [ ]:
import re
from datetime import datetime

# Define a function to load DataFrames dynamically
def load_dataframe(date_str):
    filename = f"Files/Bronze_AdventureWorks/{date_str}"
    return spark.read.option("delimiter", "\t").format("csv").option("header", "True").load(filename)

# Create an empty dictionary to store DataFrames
dataframes = {}

# Iterate over the extracted dates and load DataFrames
for item in mssparkutils.fs.ls(f'Files/Bronze_AdventureWorks/'):
    print(item.name)
    namematch = re.search(pattern, item.name)
    if namematch:
        date_str = namematch.group(1)
        df_name = f"df_{item.name.split('_')[0].lower()}"  # Create a dynamic DataFrame name
        print(date_str)
        print(df_name)
        dataframes[df_name] = load_dataframe(item.name)
        print(f"Loaded DataFrame {df_name} from {item.name}")

# Now you can access the DataFrames using dataframes['df_products'], dataframes['df_sales'], etc.


## Loading the Data

In [ ]:
df_product = spark.read.option("delimiter", "\t").format("csv").option("header", "True").load("Files/Bronze_AdventureWorks/Product_19_04_2024.csv")
df_region = spark.read.option("delimiter", "\t").format("csv").option("header", "True").load("Files/Bronze_AdventureWorks/Region_19_04_2024.csv")
df_reseller = spark.read.option("delimiter", "\t").format("csv").option("header", "True").load("Files/Bronze_AdventureWorks/Reseller_19_04_2024.csv")
df_salespersonregion = spark.read.option("delimiter", "\t").format("csv").option("header", "True").load("Files/Bronze_AdventureWorks/SalesPersonRegion_19_04_2024.csv")
df_targets = spark.read.option("delimiter", "\t").format("csv").option("header", "True").load("Files/Bronze_AdventureWorks/Targets_19_04_2024.csv")

StatementMeta(, , , Waiting, )

SynapseWidget(Synapse.DataFrame, d035de77-1fe8-424e-bc7f-e2634b393065)

## specifying datatypes
###### The [^0-9.]+ pattern used below omits all non-numeric characters and '.'

In [ ]:
from pyspark.sql.functions import col, regexp_replace, when, substring, to_date, date_format, regexp_extract
spark.conf.set("spark.sql.legacy.timeParserPolicy","LEGACY")

StatementMeta(, , , Waiting, )

In [ ]:
# PRODUCT
df_product = df_product.withColumn("ProductKey", col("ProductKey").cast("int"))
df_product = df_product.withColumn("Standard Cost", 
                                   when(col("Standard Cost").isNull(), None)
                                   .otherwise(regexp_replace(col("Standard Cost").cast("string"), '[^0-9.]+', '')))
df_product = df_product.withColumn("Standard Cost", col("Standard Cost").cast("float"))

StatementMeta(, , , Waiting, )

In [ ]:
# REGION
df_region = df_region.withColumn("SalesTerritoryKey", col("SalesTerritoryKey").cast("int"))

StatementMeta(, , , Waiting, )

In [ ]:
# RESELLER
df_reseller = df_reseller.withColumn("ResellerKey", col("ResellerKey").cast("int"))

StatementMeta(, , , Waiting, )

In [ ]:
# SALESPERSONREGION
df_salespersonregion = df_salespersonregion.withColumn("EmployeeKey", col("EmployeeKey").cast("int"))
df_salespersonregion = df_salespersonregion.withColumn("SalesTerritoryKey", col("SalesTerritoryKey").cast("int"))

StatementMeta(, , , Waiting, )

In [ ]:
# TARGETS
display(df_targets)
df_targets = df_targets.withColumn("EmployeeID", col("EmployeeID").cast("int"))
df_targets = df_targets.withColumn("Target", 
                                   when(col("Target").isNull(), None)
                                   .otherwise(regexp_replace(col("Target").cast("string"), '\\$|', '')))
df_targets = df_targets.withColumn("Target", 
                                   when(col("Target").isNull(), None)
                                   .otherwise(regexp_replace(col("Target").cast("string"), ',', '')))
df_targets = df_targets.withColumn("Target", col("Target").cast("int"))
df_targets = df_targets.withColumn("TargetMonth", to_date(col("TargetMonth"), "EEEE, MMMM dd, yyyy"))

StatementMeta(, , , Waiting, )

SynapseWidget(Synapse.DataFrame, 355f873d-f74c-4113-8427-c9d4c6be6329)

SynapseWidget(Synapse.DataFrame, 33b198e1-28d2-4a80-948c-e4e81fbfd6b0)

## Changing the column names


In [ ]:
# Product
df_product = df_product.withColumnRenamed("Standard Cost", "StandardCost")\
       .withColumnRenamed("Background Color Format", "BackgroundColorFormat")\
       .withColumnRenamed("Font Color Format", "FontColorFormat")\
       .withColumnRenamed("Product", "ProductName")\
       .withColumnRenamed("ProductKey", "ProductID")

StatementMeta(, , , Waiting, )

In [ ]:
# Reseller
df_reseller = df_reseller.withColumnRenamed("Business Type", "BusinessType")\
    .withColumnRenamed("State-Province", "StateProvince")\
    .withColumnRenamed("Country-Region", "CountryRegion")\
    .withColumnRenamed("ResellerKey", "ResellerID")
display(df_reseller)

StatementMeta(, , , Waiting, )

SynapseWidget(Synapse.DataFrame, 81bac401-4ef7-4b07-81de-fdf0debabd99)

In [ ]:
# Write the DataFrame to Parquet format in the silver lakehouse
# "/Product" is the delta table name
df_product.write.format("delta").mode("overwrite").save(f"{silver_tables_path}/Product")
df_region.write.format("delta").mode("overwrite").save(f"{silver_tables_path}/Region")
df_reseller.write.format("delta").mode("overwrite").save(f"{silver_tables_path}/Reseller")
df_salespersonregion.write.format("delta").mode("overwrite").save(f"{silver_tables_path}/SalesPersonRegion")
df_targets.write.format("delta").mode("overwrite").save(f"{silver_tables_path}/Targets")

StatementMeta(, , , Waiting, )

In [ ]:
df_product.printSchema()
df_region.printSchema()
df_reseller.printSchema()
df_salespersonregion.printSchema()
df_targets.printSchema()

StatementMeta(, , , Waiting, )

root
 |-- ProductKey: integer (nullable = true)
 |-- Product: string (nullable = true)
 |-- StandardCost: float (nullable = true)
 |-- Color: string (nullable = true)
 |-- Subcategory: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- BackgroundColorFormat: string (nullable = true)
 |-- FontColorFormat: string (nullable = true)

root
 |-- SalesTerritoryKey: integer (nullable = true)
 |-- Region: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Group: string (nullable = true)

root
 |-- ResellerKey: integer (nullable = true)
 |-- BusinessType: string (nullable = true)
 |-- Reseller: string (nullable = true)
 |-- City: string (nullable = true)
 |-- StateProvince: string (nullable = true)
 |-- CountryRegion: string (nullable = true)

root
 |-- EmployeeKey: integer (nullable = true)
 |-- SalesTerritoryKey: integer (nullable = true)

root
 |-- EmployeeID: integer (nullable = true)
 |-- Target: integer (nullable = true)
 |-- TargetMonth: date (nullable